# Genesis4 Resistive Wall Wakefield

Parameters from SLAC-PUB-10707

Note that we use a Gaussian here instead of a double-horn distribution.

For comparison, we choose a copper pipe with a 2.5 mm radius, and track for 1 m.

### Table I: Selected Parameters in LCLS Undulator

| Parameter              | Symbol | Value     | Unit   |
|------------------------|--------|-----------|--------|
| Beam energy            | E      | 14        | GeV    |
| Bunch charge           | eN     | 1         | nC     |
| RMS bunch length       | σz     | 20        | µm     |
| Beam pipe radius       | a      | 2.5       | mm     |
| Beam pipe length       | L      | 130       | m      |


In [ ]:
# import os
from math import pi, sqrt

import matplotlib.pyplot as plt
from scipy.constants import c

import genesis.version4 as g4

from pmd_beamphysics.units import mec2

In [ ]:
# Parameters
RADIUS = 2.5e-3  # m
ENERGY0 = 14e9  # eV
SIGMA_Z = 20e-6  # m
BUNCH_CHARGE = 1e-9  # C


SIGMA_T = SIGMA_Z / c
PEAK_CURRENT = BUNCH_CHARGE / (sqrt(2 * pi) * SIGMA_T)
SLEN = 10 * SIGMA_Z
S0 = 5 * SIGMA_Z
SIGMA_T, SIGMA_Z, SLEN

In [ ]:
# Lattice
D1 = g4.Drift(L=1)
lattice = g4.Lattice(elements={"D1": D1, "LAT": g4.Line(elements=[D1])})

In [ ]:
# Main input
gamma0 = ENERGY0 / mec2

main = g4.MainInput(
    namelists=[
        g4.Setup(
            rootname="SLAC-PUB-10707",
            beamline="LAT",
            gamma0=gamma0,
            lambda0=1e-7,  # arbitrary for this purpose
            # delz=0.026,
            seed=123456,
            npart=128,
        ),
        g4.Time(slen=SLEN),
        g4.ProfileGauss(
            label="beamcurrent",
            c0=PEAK_CURRENT,
            s0=S0,
            sig=SIGMA_Z,
        ),
        g4.Beam(
            gamma=gamma0,
            delgam=0.0001,  # very small to see the wake
            current="beamcurrent",
        ),
        g4.Wake(
            radius=2.5e-3,
            material="CU",
            roundpipe=True,  # default
            # conductivity=6.5e7,    # From SLAC-PUB-10707
            # relaxation=27e-15 * c, # From SLAC-PUB-10707
        ),
        g4.Write(beam="beginning"),
        g4.Track(zstop=1),
        g4.Write(beam="end"),
    ],
)

G = g4.Genesis4(main, lattice, verbose=False)
output = G.run()

In [ ]:
!cat {G.path}/genesis4.in

In [ ]:
output.load_particles()
P0 = output.particles["beginning"]
P1 = output.particles["end"]
P1.plot("delta_z/c", "energy")
P1

In [ ]:
# Mean energy loss
e1 = P1["mean_energy"]
e0 = P0["mean_energy"]
e1 - e0  # energy change in eV

In [ ]:
plt.scatter(P1.z / c * 1e15, P1.energy - ENERGY0, marker=".")
plt.xlabel(r"$z/c$ (fs)")
plt.ylabel(r"$\Delta E$ (eV)")